### Import Required Libraries and Set Up Environment Variables

In [2]:
!pip install python-dotenv
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [4]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# query_url = url + "api-key=" + nyt_api_key + "&q=" + filter_query
# query_url = f"{url}api-key={nyt_api_key}&q={filter_query}&begin_date={begin_date}&end_date={end_date}"
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
# create query with a page number
# API results show 10 articles at a time
for page in range(0, 20):
    
    # extend query with page number
    query_url_page = f"{query_url}&page={str(page)}"
     
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url_page).json()
    # print(reviews)
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # # Try and save the reviews to the reviews_list
    # # loop through the reviews["response"]["docs"] and append each review to the list
    for review in reviews["response"]["docs"]:
        try:
            reviews_list.append(review)
            print(f"{review['headline']['print_headline']} found! Appending review. Page {page}")
        
    # # Handle exceptions for a reviews that are not available in the NYT API
        except:
            # Print exception note
            print(f"No reviews found on page {page}.")
            # pass
            break

    
        
        
    #     # Print the page that was just retrieved      
    #     print(review['snippet'])
    #     print('---------------------------')

    #     # Print the page number that had no results then break from the loop


The Attachment Diaries found! Appending review. Page 0
What’s Love Got to Do With It? found! Appending review. Page 0
You Can Live Forever found! Appending review. Page 0
A Tourist’s  Guide to Love found! Appending review. Page 0
Intoxicating Love With a Sobering Turn found! Appending review. Page 0
One True Loves found! Appending review. Page 0
The Lost Weekend:  A Love Story found! Appending review. Page 0
An Unbending Will Meets a Shifting City found! Appending review. Page 0
They Have a Humdrum Kind of Love found! Appending review. Page 0
Love in the Time Of Fentanyl found! Appending review. Page 0
Pamela, a Love Story found! Appending review. Page 1
In From the Side found! Appending review. Page 1
After Love found! Appending review. Page 1
Alcarràs found! Appending review. Page 1
Nelly &amp; Nadine found! Appending review. Page 1
A Love Whose Name Is Often Spoken found! Appending review. Page 1
The Sound  Of Christmas found! Appending review. Page 1
A Few Good Men, Some With Secre

In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
json_string = json.dumps(reviews_list, indent=4)
sliced_json_string = json_string[:json_string.index("One True Loves")]
print(sliced_json_string)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
results_df = pd.json_normalize(reviews_list)
results_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [8]:
# have a peek at the column names
results_df.columns


Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [9]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
results_df['title'] = results_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
results_df['title'] 

0                             The Attachment Diaries
1      What’s Love Got to Do With It?’ Probably a Lo
2                               You Can Live Forever
3                          A Tourist’s Guide to Love
4                            Other People’s Children
                           ...                      
195                                   The Other Half
196                           The Ottoman Lieutenant
197                                     Love & Taxes
198    Everybody Loves Somebody,’ a Rom-Com With Bit
199                Kedi,’ Rekindling a ‘Love of Life
Name: title, Length: 200, dtype: object

In [10]:
# defined function to extract key words
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # print(keyword)
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Extract 'name' and 'value' from items in "keywords" column
results_keywords_df = []
results_keywords_df = results_df['keywords'].apply(extract_keywords)
results_keywords_df = pd.DataFrame(results_keywords_df, columns=['keywords'])
# print(results_keywords_df)

# Fix the "keywords" column by converting cells from a list to a string
results_keywords_df['keywords'] = results_keywords_df['keywords'].astype(str)
results_keywords_df['keywords']
# flattened_string = ""
# for dictionary in results_keywords_df['keywords']:
#     for key, value in dictionary.items():
#         flattened_string += f"{key}: {value}, "

# # Remove the trailing comma and space
# flattened_string = flattened_string[:-2]

# # Print the flattened string
# print(flattened_string)

# results_keywords_df['keywords'] = flattened_string

0      subject: Movies;creative_works: The Attachment...
1      subject: Movies;persons: Kapur, Shekhar;person...
2      subject: Movies;creative_works: You Can Live F...
3      subject: Movies;creative_works: A Tourist's Gu...
4      subject: Movies;persons: Zlotowski, Rebecca;cr...
                             ...                        
195    subject: Movies;creative_works: The Other Half...
196    subject: Movies;creative_works: The Ottoman Li...
197    creative_works: Love & Taxes (Movie);persons: ...
198    subject: Movies;creative_works: Everybody Love...
199    creative_works: Kedi (Movie);subject: Document...
Name: keywords, Length: 200, dtype: object

In [11]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = []
title_list = results_df['title'].to_list()
print(title_list)

['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

### Access The Movie Database API

In [12]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


# # Documentation: https://developer.themoviedb.org/docs/search-and-query-for-details
# query_url = "https://api.themoviedb.org/3/search/movie?query="

# title = "Everything Everywhere All at Once"

# # Execute "GET" request with url
# response_data = requests.get(query_url + title + "&api_key=" + api_key)

In [158]:
# Create an empty list to store the reviews
tmdb_movies_list = []
tmdb2_api_results_first_five = []
movie_id = ""

# Create a request counter to sleep the requests after a multiple
# of 50 requests
sleep_counter = 50
request_counter = 1
num_sleeps = 0

# Loop through the titles
for titles in title_list:

    # Check if we need to sleep before making a request
    if (request_counter % sleep_counter) == 0:
        time.sleep(1)
        num_sleeps += 1
        print(f"Sleep interval #: {num_sleeps}")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database based on titles
    tmdb_result = requests.get(url + titles + tmdb_key_string).json()

    # Try to extract data out of the API results
    try:
        # Extract movie ID
        movie_id = tmdb_result["results"][0]['id']

        # Append movie ID
        tmdb_movies_list.append(movie_id)

        # Create new URL for second API search based on movie ID
        url2 = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Perform a "GET" request for The Movie Database based on movie ID
        tmdb2_api_results = requests.get(url2).json()
       
        # Extract genre names into list
        genres = []
        for genre in tmdb2_api_results['genres']:
            genres.append(genre['name'])

        # Extract the spoken languages English name
        spoken_languages = []
        for languages in tmdb2_api_results['spoken_languages']:
            spoken_languages.append(languages['english_name'])

        # Extract the production countries
        production_countries = []
        for countries in tmdb2_api_results['production_countries']:
            production_countries.append(countries['name'])

        # Define the temp dictionary for this movie
        tmdb_dictionary = {'title':[], 'original_title':[], 'budget':[], 'original_language':[], 'homepage':[], 'overview':[], 'popularity':[], 'runtime':[], 'revenue':[], 'release_date':[],\
                   'vote_average':[], 'vote_count':[], 'generes':[], 'spoken_language':[], 'production_countries':[]}

        # Extract values from the API results into the temp movie dictionary 
        tmdb_dictionary['title'] = tmdb2_api_results['title']
        tmdb_dictionary['original_title'] = tmdb2_api_results['original_title']
        tmdb_dictionary['budget'] = tmdb2_api_results['budget']
        tmdb_dictionary['original_language'] = tmdb2_api_results['original_language']
        tmdb_dictionary['homepage'] = tmdb2_api_results['homepage']
        tmdb_dictionary['overview'] = tmdb2_api_results['overview']
        tmdb_dictionary['popularity'] = tmdb2_api_results['popularity']
        tmdb_dictionary['runtime'] = tmdb2_api_results['runtime']
        tmdb_dictionary['revenue'] = tmdb2_api_results['revenue']
        tmdb_dictionary['release_date'] = tmdb2_api_results['release_date']
        tmdb_dictionary['vote_average'] = tmdb2_api_results['vote_average']
        tmdb_dictionary['vote_count'] = tmdb2_api_results['vote_count']
        tmdb_dictionary['generes'] = genres
        tmdb_dictionary['spoken_language'] = spoken_languages
        tmdb_dictionary['production_countries'] = production_countries

        # Append the temp dictionary for this movie into to the dictionary of movie results
        tmdb_movies_list.append(tmdb_dictionary)

        # For any successful attempts on the API result extractions:
        # print(f"{titles} was found.")
                
    # For any failed attempts on the API result extractions:
    except:    
        # show "no results" in the temp dictionary for this movie
        tmdb_dictionary = {'title':[], 'original_title':[], 'budget':[], 'original_language':[], 'homepage':[], 'overview':[], 'popularity':[], 'runtime':[], 'revenue':[], 'release_date':[],\
                   'vote_average':[], 'vote_count':[], 'generes':[], 'spoken_language':[], 'production_countries':[]}
        # print(f"{titles} NOT found.")
    
    # Capture the first 5 json data results in a list
    if request_counter <= 6:   
        print(tmdb_dictionary)
        tmdb2_api_results_first_five.append(tmdb2_api_results)

{'title': 'The Attachment Diaries', 'original_title': 'El apego', 'budget': 0, 'original_language': 'es', 'homepage': '', 'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.', 'popularity': 1.278, 'runtime': 102, 'revenue': 0, 'release_date': '2021-10-07', 'vote_average': 3.0, 'vote_count': 4, 'generes': ['Drama', 'Mystery', 'Thriller', 'Horror'], 'spoken_language': ['Spanish'], 'production_countries': ['Argentina']}
{'title': [], 'original_title': [], 'budget': [], 'original_language': [], 'homepage': [], 'overview': [], 'popularity': [], 'runtime': [], 'revenue': [], 'release_date': [], 'vote_average': [], 'vote_count': [], 'generes': [], '

In [159]:
# Convert the results to a dataframe
# tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df = pd.json_normalize(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,generes,spoken_language,production_countries
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Attachment Diaries,El apego,0.0,es,,"Argentina, 1970s. A desperate young woman goes...",1.278,102.0,0.0,2021-10-07,3.000,4.0,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,You Can Live Forever,You Can Live Forever,0.0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",21.801,96.0,15055.0,2023-03-24,6.474,38.0,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,The Other Half,The Other Half,0.0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,6.874,103.0,0.0,2016-12-02,6.300,24.0,"[Drama, Romance]",[English],[Canada]
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000.0,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",13.046,111.0,413844.0,2017-03-10,6.164,247.0,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"
306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
# Preview the first 5 results in JSON format
for i in tmdb2_api_results_first_five:
    
        # print the json data
        tmdb2_json_string = json.dumps(i, indent=4)
        print(tmdb2_json_string)

{
    "adult": false,
    "backdrop_path": "/r2Ua7wh6MAdu1PAyCb045VGjMv9.jpg",
    "belongs_to_collection": null,
    "budget": 0,
    "genres": [
        {
            "id": 18,
            "name": "Drama"
        },
        {
            "id": 9648,
            "name": "Mystery"
        },
        {
            "id": 53,
            "name": "Thriller"
        },
        {
            "id": 27,
            "name": "Horror"
        }
    ],
    "homepage": "",
    "id": 743040,
    "imdb_id": "tt11359158",
    "origin_country": [
        "ES"
    ],
    "original_language": "es",
    "original_title": "El apego",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange a

In [ ]:
print(response_data)

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

# json_string = json.dumps(reviews_list, indent=4)
# sliced_json_string = json_string[:json_string.index("One True Loves")]
# print(sliced_json_string)

In [ ]:
# Convert the results to a DataFrame

# results_df = pd.json_normalize(reviews_list)
# results_df

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
